In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.cluster import KMeans, AgglomerativeClustering
import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import time
import h5py
import pickle
import csv

In [2]:
df = pd.read_csv("/Users/jpate/Downloads/netflix_titles.csv")

In [3]:
df = df.dropna(axis = 0)

In [4]:
df = df[df["release_year"]>=2000]

In [5]:
df = df[(df["rating"] != "NC-17") & (df["rating"] != "UR") & (df["rating"] != "TV-Y7-FV")]

In [6]:
df = df.reset_index(drop=True)

In [7]:
df.to_csv('idk.csv')

In [8]:
df_titles = df.copy()

In [9]:
df_titles = df_titles.drop(['show_id','director','cast','country','date_added', 'release_year','type','rating','duration','listed_in','description'], axis=1)

In [10]:
titles = df_titles.to_dict()

In [11]:
titles = titles['title']

In [13]:
nlp = spacy.load('en_core_web_lg')

In [124]:
similarities = {}

In [31]:
def func():
    for i, row in df.iterrows():
        doc = nlp(row['description'])
        #similarities[i] = {}
        test[i]={}
#     print(row['title'])
        for j, row2 in df.iterrows():
            doc2 = nlp(row2['description'])
        #print(f"{row['title']} x {row2['title']}: {doc.similarity(doc2):.10f}")
        #similarities[i][j] = doc.similarity(doc2)
            test[i][j] = doc.similarity(doc2)

In [19]:
def lookup_by_title(title):
    for title_id in titles:
        if titles[title_id] == title:
            return title_id
    return None

In [20]:
def lookup(title1, title2):
    return similarities[lookup_by_title(title1)][lookup_by_title(title2)]

In [370]:
import pickle

# define dictionary

# create a binary pickle file 
f = open("dict.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(new_dict,f)

# close file
f.close()

In [14]:
infile = open('dict.pkl','rb')
similarities = pickle.load(infile)
infile.close()

In [32]:
func()

TypeError: __call__() takes 2 positional arguments but 3 were given

In [21]:
from heapq import nlargest
def recommend(item_title):
    six_largest = nlargest(6, 
                            similarities[lookup_by_title(item_title)], 
                            key=similarities[lookup_by_title(item_title)].get)
    six_largest.pop(0)
    for x in six_largest:
        print(titles[x])

In [23]:
recommend('The Great British Baking Show')

Shopkins: Chef Club
Belgica
Tune in for Love
Berlin Calling
Maacher Jhol
